# MNIST using simple ANN baseline model

In this notebook , we are going to go through the details on how to build a simple deep learning model (ANN) to predict the labels of handwritten digits given it's image . We would be using MNIST dataset from Kaggle for training the model ([link](https://www.kaggle.com/competitions/digit-recognizer)) .I'll try my best to keep things as simple as possible here and explain the steps and the process we follow as we go through this notebook . 
The first step is to inspect the dataset, the format in which the images are stored and the correspoinding labels assciated with the same .  

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We notice that the data is stored in the form of csv . We can quickly use pandas to read the CSVs in the form of dataframe and go though it's contents . We read through both the train and test files to see the columns in each of them and that'll give us an idea on what the target variable is . 

In [6]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(train_df.shape , test_df.shape)
print("Train Cols : " , train_df.columns)
print("Test Cols : " , test_df.columns)
print(" Label -  " , [i for i in train_df.columns if i not in test_df.columns] )

There are 42K Rows in the training dataset and 28K in the test_dataset . From the column names , we conculde that the pictures are stored in the form of rows where the column - <b> pixel_1 </b> indicates the pixel density value in the <b> cell_no[1] </b> in the image  . Since there are values till pixel_783 , there are 784 pixel counts which means it's a 28*28 image . To find the target column , we simply check for columns in the training dataset that is not a part of test dataset

## Creating the dataset class for the pytorch model

We are going to follow best practices of pytorch get the data in the form of <b>Dataset</b> since it makes life easier for us to create the dataloaders which would later be used in the training processes . Here we create our own custom class inheriting the <b> Dataset </b> class specifying how to access the elements of the dataset undet the __getitem__ method . Under this method , we read the data and store it under __img_df__ and extract the image , convert them into 28*28 numpy array  , bring the values under the range [0,1] and perform transforms on both the image and the label seperately and returns them in the form of X , y tensors . This custom class accepts the following arguments
* csv_name - The name under which the data is stored 
* img_dir - the path of the directory under which the file is stored.
* transform - The tranforms that needs to be done on the image vector 
* target_transform - The transforms that needs to be done on the target variable

In [9]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor , Lambda

class CustomMNISTDataset(Dataset):
    def __init__(self, csv_name, img_dir, transform=None, target_transform=None , label_name = "label"):
        
        self.img_filename = csv_name
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.label_name = label_name
        
        img_path = os.path.join(self.img_dir, self.img_filename)
        self.img_df = pd.read_csv(img_path)

    def __len__(self):
        return len(self.img_df)

    def __getitem__(self, idx):
        
        # Extracting all the other columns except label_name
        img_cols = [ i for i in self.img_df.columns if i not in self.label_name]
        
        image = self.img_df.iloc[[idx]][img_cols].values
        label = int(self.img_df.iloc[[idx]][self.label_name].values)
        
        # Reshaping the array from 1*784 to 28*28
        
        image = image.reshape(28,28)
        # image = image.astype(float)
        
        # Scaling the image so that the values only range between 0 and 1
        image = image/255.0
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        image = image.to(torch.float)
        # label = label.to(torch.float)
        
        return image, label

### Distribution of target labels

Before actually creating the datasets , we just peek into the training dataset just to see how many target variables are actually present in the dataset . This will give us a sense of whether the dataset is skewed towards a particular label compared to others

In [7]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_df['label'].value_counts().sort_index()

We see that the distribution of the labels are fairly equal and is not highly skewed towards any particular varibale here.

### Distribution of labels in both train and valid sets

To measure at the actual model performance , it is required to create train and valid subsets from our training_data set . We will only use the train_subset for training the model and will check the performance of the same with the valid dataset . For creating the valid dataset , we will stratify using the y-labels just to ensure that the distribution of both the train and valid subsets remain the same

In [12]:
## Illustration of creating a validation set 

from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))

train_indices , test_indices = train_test_split(indices, test_size=0.1, stratify=train_df['label'])
# train_indices , test_indices = train_test_split(indices, test_size=0.1)

len(train_indices) , len(test_indices) , len(train_df)

train_subset = train_df.loc[train_indices]
val_subset = train_df.loc[test_indices]

print("Distribution of target values in training dataset ; ")
print( train_subset['label'].value_counts().sort_index() / train_subset['label'].value_counts().sort_index().sum() )

print("Distribution of target values in validation dataset ; ")
print( val_subset['label'].value_counts().sort_index() / val_subset['label'].value_counts().sort_index().sum() )

## Creating Train Dataset using train data

We will go ahead and create the train_dataset using the class we have defined above . As for the arguments , we supply the directory and the csv_name based on the values provided below . For transforms , we use the function ToTensor() which converts the numpy aray to tensor and Normalize(mean , std) helping us normalize the dataset with the given mean and standard deviation so that the values are not out pf proportion . We keep the target_transform value as None since nothing needs to be done on the target variable

In [13]:
from torchvision import transforms

# Crerating a temp dataset
train_csv_name = "train.csv"
test_csv_name = "test.csv"
img_dir = "/kaggle/input/digit-recognizer/"

# Converting X variables to Tensors
transforms = transforms.Compose( [transforms.ToTensor() , transforms.Normalize((0.5,), (0.5,)) , ] )

# Converting y-labels to one hot encoding
# target_transform = Lambda(lambda y: torch.zeros(
#     len(train_df['label'].unique()), dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
# This is not need since we are going to be using cross entropy loss function

label_name = "label"

train_dataset = CustomMNISTDataset(csv_name = train_csv_name , img_dir = img_dir , transform = transforms , target_transform = None , label_name = label_name)

# Inspecting the fist line item under dataset
x0 , y0 = train_dataset[0]
print(x0.shape , y0)

### Plotting the datapoints of train_dataset

In [17]:
# Ploting some of the datapoints in the dataset
import matplotlib.pyplot as plt

# sample_img , sample_lbl = temp_train_dataset[3]
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
figure.add_subplot(rows, cols, 1)
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
    sample_img , sample_lbl = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(sample_lbl)
    plt.axis("off")
    plt.imshow(sample_img.squeeze(), cmap="gray")
plt.show()

Checking if the device has GPU present , if not we do the whole training process on CPU (The downside being it can be very slow)

In [ ]:
## Checking if the GPU is being used properly . 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.cuda.is_available()
x0 = x0.to(device)
print("x0" , x0.is_cuda)

## Creating test and valid subsets using stratified samples

We then proceed on splitting the training_dataset into train and valid subsets as defined above using stratified sampling . We keep the valid set as 10% of the training_dataset .Notice that when we look at the contents of train_dataloaders , the dimension is (torch.Size([64, 1, 28, 28]), torch.Size([64])) which is differnt from the contents of train_dataset torch.Size([1, 28, 28]), int). This is essentially a batch that gets processed at the time of training .

In [18]:
from torch.utils.data import SubsetRandomSampler
from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))
train_indices , valid_indices = train_test_split(indices, test_size=0.1, stratify=train_df['label'])

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(test_indices)

train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=64, sampler=train_sampler, num_workers=16)
valid_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=valid_sampler, num_workers=16)

x0 , y0 = next(iter(train_dataloader))
x0.shape , y0.shape

## Defining NN Model 

Now that we are finished with defining the dataloaders , we then proceed to defining the details of the Neural Netowrk are going to be using for predictions. We use an architecture with 2 hidden layers and with the output layer having 10 Nodes (since there are 10 differnt classes for prediciton) . We define our our class inheriting the nn.Module to define our own forward propagation function since it's considered as best practice while working on pytorch .

The model would look as follows :

* We flatten our 28*28 image into a tensor of length 784 (which is achieved through nn.Flatten)
* We then input these into a hidden layer containing 128 nodes which is then connected to another hidden layer with 64 nodes. We use Relu as activation function inbetween layers 
* Finally we then connect the hidden layer with a layer having 10 nodes (Equivalent to the number of labels) 

<p> Notice that there's no softmax layer at the end of the NN . This is because the <b> nn.CrossEntropyLoss() </b> automatically applies softmax from the output obtained to calculate loss . However , if we use <b> nn.NLLLLoss() </b> as our loss fucntion , then we would have to include <b> nn.LogSoftmax </b> at the end of nn.Sequential(..) </p>

In [19]:
from torch import nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class MyOwnNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyOwnNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
            ## Softmax layer ignored since the loss function defined is nn.CrossEntropy()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return  logits
    
    
model = MyOwnNeuralNetwork().to(device)
print(model)

# model = model.cuda()
# torch.backends.cudnn.benchmark=True
# torch.cuda.set_device(0)

## Defining Optimizers and loss function

We will use cross entrpy loss as loss function and stocastic gradient descent as optimizer for this particular exercise

In [20]:
## Defining optimizer and loss functions 

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=3e-3, momentum=0.9)

## Defining Train loop

In [ ]:
from torch.autograd import Variable
def train(dataloader, model, loss_fn, optimizer):
    
    # Total size of dataset for reference
    size = len(dataloader.dataset)
    
    # places your model into training mode
    model.train()
    
    correct = 0
    # Gives X , y for each batch
    for batch, (X, y) in enumerate(dataloader):
        
        # Converting device to cuda
        X, y = X.to(device), y.to(device)
        model.to(device)
        
        # Compute prediction error / loss
        # 1. Compute y_pred 
        # 2. Compute loss between y and y_pred using selectd loss function
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation on optimizing for loss
        # 1. Sets gradients as 0 
        # 2. Compute the gradients using back_prop
        # 3. update the parameters using the gradients from step 2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    correct/=size
    print(f"Train Accuracy: {(100*correct):>0.1f}%")

## Defining valid/test loop

In [ ]:
def validation(dataloader, model, loss_fn):
    
    # Total size of dataset for reference
    size = 0
    num_batches = len(dataloader)
    
    # Explanation given above
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        
        # Gives X , y for each batch
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            model.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            size+=len(y)
    
    ## Calculating loss based on loss function defined
    test_loss /= num_batches
    
    ## Calculating Accuracy based on how many y match with y_pred
    print(size)
    correct /= size
    
    print(f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Training and validation triggers 

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    validation(valid_dataloader, model, loss_fn)
print("Done!")